In [1]:
import pandas as pd
import dash
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import json
import geojson


df_dejavnosti = pd.read_csv("data/podatki_po_dejavnostih.csv", delimiter=",", encoding="windows-1250")
df_odjemalci = pd.read_csv("data/podatki_vrsta_odjemalca.csv", delimiter=",", encoding="windows-1250")

In [37]:
map_df = gpd.read_file('slovenija_map/obcine/obcine.shp')

len(map_df)

212

In [35]:
fig = px.choropleth(map_df, geojson=data, color="rand",
                    locations="District", featureidkey="properties.District",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [57]:
df_odjemalci_backup = df_odjemalci.copy()

#Tukaj vzamemo samo podatke za VSE SKUPAJ porabo (brez gospodinjstva in poslovnih objektov)
df = df_odjemalci_backup[::+3]
#Tukaj izbrišem podatke za SLOVENIJO in Neznano vrstice
df = df[1:-1]

#Priprava podatkov za uporabo
df = df.drop("VRSTA ODJEMALCA", axis=1)
obcine = {df["OBČINE"][i]:i for i in df.index}

vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
vse_2022 = dict(df[df.columns[25:34]].sum(axis=1))
dict_porab = dict([(k,[vse_2020[k], vse_2021[k], vse_2022[k]]) for k in vse_2020])

#V koncem dictu so shranjene OBČINE (ključ) z vrednostmi, ki so LISTI, 
# ki prikazujejo porabo za vsako občino za vsako leto posebaj
koncni_dict = dict()
for k,v in obcine.items():
    koncni_dict[k] = dict_porab[v]

koncni_dict

{'Ajdovščina': [93181950, 100935906, 77325837],
 'Ankaran/Ancarano': [19112313, 19604562, 15257254],
 'Apače': [10051617, 10623990, 7368970],
 'Beltinci': [24589761, 24562474, 16390906],
 'Benedikt': [6607746, 6991660, 5058807],
 'Bistrica ob Sotli': [3319459, 3620303, 2626785],
 'Bled': [38934450, 40729710, 33239772],
 'Bloke': [6787877, 6806006, 4722042],
 'Bohinj': [28971836, 31605428, 24331600],
 'Borovnica': [19832338, 21052894, 15322526],
 'Bovec': [18568541, 20577552, 16059663],
 'Braslovče': [16420394, 17039269, 11806901],
 'Brda': [18503852, 19474588, 15173245],
 'Brezovica': [33368066, 35260547, 25182459],
 'Brežice': [90200634, 96509162, 72781771],
 'Cankova': [5074799, 5370870, 3832400],
 'Celje': [330313558, 339003018, 251599120],
 'Cerklje na Gorenjskem': [43976033, 48939544, 36221651],
 'Cerknica': [49833430, 53790308, 38575318],
 'Cerkno': [39151682, 43206346, 30710422],
 'Cerkvenjak': [5421889, 5720693, 4201221],
 'Cirkulane': [6658585, 6972246, 4998396],
 'Črenšovci':

In [ ]:
x = list(koncni_dict.keys())
y = list()

for value in koncni_dict.values():
    y.append(value[0])

plt.figure(figsize=(15,10))
plt.barh(x, y, color='#F72585', edgecolor='black')
plt.grid(axis='x')
plt.xticks(np.arange(0, 1000000000, step=1000000))
plt.title('Poraba elektrike po občinah v letu 2020', fontsize=20)
plt.xlabel('Ime občine', fontsize=15)
plt.ylabel('Poraba elektrike', fontsize=15)
plt.show()
